# 第二章、向量数据库的介绍及使用

 - [一、向量数据库简介](#一、向量数据库简介)
 - [二、构建 Chroma 向量库](#二、构建-Chroma-向量库)
 - [三、通过向量数据库检索](#三、通过向量数据库检索)
     - [3.1 相似度检索](#3.1-相似度检索)
     - [3.2 MMR 检索](#3.2-MMR-检索)
 - [四、构造检索式问答连](#四、构造检索式问答连)
     - [4.1 直接询问 LLM](#4.1-直接询问-LLM)
     - [4.2 结合 prompt 提问](#4.2-结合-prompt-提问)


## 一、向量数据库简介

向量数据库是用于高效计算和管理大量向量数据的解决方案。向量数据库是一种专门用于存储和检索向量数据（embedding）的数据库系统。它与传统的基于关系模型的数据库不同，它主要关注的是向量数据的特性和相似性。

在向量数据库中，数据被表示为向量形式，每个向量代表一个数据项。这些向量可以是数字、文本、图像或其他类型的数据。向量数据库使用高效的索引和查询算法来加速向量数据的存储和检索过程。

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

In [1]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

In [ ]:
# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env fileopenai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key  = os.environ['OPENAI_API_KEY']
zhihuai.api_key = os.environ['ZHIPUAI_API_KEY']

In [3]:
# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("../../docs/knowledge_base/LeeDL_Tutorial.pdf") # 机器学习,
    # PyMuPDFLoader("../../docs/knowledge_base/EasyRL.pdf") # 强化学习,
    # PyMuPDFLoader("../../docs/knowledge_base/LLM_Survey_Chinese.pdf") # 大语言模型
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)


# 定义 Embeddings
# embedding = OpenAIEmbeddings() 
# embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
embedding = ZhipuAIEmbeddings()

In [4]:
persist_directory = '../../docs/chroma/knowledge_base'

In [20]:
!rm -rf '../../docs/chroma/knowledge_base'  # 删除旧的数据库文件（如果文件夹中有文件的话），window电脑请手动删除

## 二、构建 Chroma 向量库

In [21]:
vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [22]:
vectordb.persist()

大家也可以直接载入已经构建好的向量库

In [5]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [24]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：100


## 三、通过向量数据库检索

### 3.1 相似度检索

In [6]:
question="什么是机器学习"

In [7]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [12]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
→_→
https://github.com/datawhalechina/leedl-tutorial
←_←
第 1 章
机器学习基础
首先简单介绍一下机器学习（machine learning）和深度学习（deep learning）的基本概
念。机器学习，顾名思义，机器具备有学习的能力。具体来讲，机器学习就是让机器具备找一
个函数的能力。机器具备找函数的能力以后，它可以做很多事，举个例子：
--------------
检索到的第1个内容: 
器不只是要做选择题或输出一个数字，而是产生一个有结构的物体，比如让机器画一张图，写
一篇文章。这种叫机器产生有结构的东西的问题称为结构化学习。
1.1
案例学习
以视频的点击次数预测为例介绍下机器学习的运作过程。假设有人想要通过视频平台赚
钱，他会在意频道有没有流量，这样他才会知道他的获利。假设后台可以看到很多相关的信
息，比如：每天点赞的人数、订阅人数、观看次数。根据一个频道过往所有的信息可以预
--------------
检索到的第2个内容: 
前言
李宏毅老师是台湾大学的教授，其《机器学习》（2021 年春）是深度学习领域经典的中文视
频之一。李老师幽默风趣的授课风格深受大家喜爱，让晦涩难懂的深度学习理论变得轻松易懂，
他会通过很多动漫相关的有趣例子来讲解深度学习理论。李老师的课程内容很全面，覆盖了到深
度学习必须掌握的常见理论，能让学生对于深度学习的绝大多数领域都有一定了解，从而可以进
一步选择想要深入的方向进行学习，对于想入门深度学
--------------


### 3.2 MMR 检索

如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。


核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [14]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [15]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
→_→
https://github.com/datawhalechina/leedl-tutorial
←_←
第 1 章
机器学习基础
首先简单介绍一下机器学习（machine learning）和深度学习（deep learning）的基本概
念。机器学习，顾名思义，机器具备有学习的能力。具体来讲，机器学习就是让机器具备找一
个函数的能力。机器具备找函数的能力以后，它可以做很多事，举个例子：
--------------
MMR 检索到的第1个内容: 
器不只是要做选择题或输出一个数字，而是产生一个有结构的物体，比如让机器画一张图，写
一篇文章。这种叫机器产生有结构的东西的问题称为结构化学习。
1.1
案例学习
以视频的点击次数预测为例介绍下机器学习的运作过程。假设有人想要通过视频平台赚
钱，他会在意频道有没有流量，这样他才会知道他的获利。假设后台可以看到很多相关的信
息，比如：每天点赞的人数、订阅人数、观看次数。根据一个频道过往所有的信息可以预
--------------
MMR 检索到的第2个内容: 
15.4 元学习的应用 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 279
第 16 章 终生学习
281
16.1 灾难性遗忘 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 281
16.
--------------


可以看到内容有了更多的差异。

## 四、构造检索式问答连

我们已经可以通过向量数据库找到最相关的内容了，接下来我们可以让 LLM 来用这些相关的内容回答我们的问题。

### 4.1 直接询问 LLM

基于 LangChain，我们可以构造一个使用 LLM 进行问答的检索式问答链，这是一种通过检索步骤进行问答的方法。我们可以通过传入一个语言模型和一个向量数据库来创建它作为检索器。然后，我们可以用问题作为查询调用它，得到一个答案。

In [16]:
# 导入检索式问答链
from langchain.chains import RetrievalQA

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# 可以使用 HuggingFacePipeline 本地搭建大语言模型
model_id = 'HUDM/chatglm2-6b'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, trust_ remote_ code-True).half().quantize(4).cuda()
model = model.eval()
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

llm = HuggingFacePipeline(pipeline=pipe)

In [18]:
llm = ZhipuAILLM(model="chatglm_std", temperature=0)

In [19]:
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [20]:
# 可以以该方式进行检索问答
question = "本知识库主要包含什么内容"
result = qa_chain({"query": question})
print(f"大语言模型的回答为：{result['result']}")

大语言模型的回答为：" 本知识库主要包含了机器学习和深度学习的相关知识，如基本概念、分类、回归、结构化学习等。通过详细的推导过程、重点讲解和强化，以及丰富的例子，使得深度学习的理论变得轻松易懂。此外，本知识库还涵盖了除公开课之外的其他深度学习相关知识，以帮助读者更好地理解和入门深度学习。"


### 4.2 结合 prompt 提问

对于 LLM 来说，prompt 可以让更好的发挥大模型的能力。


我们首先定义了一个提示模板。它包含一些关于如何使用下面的上下文片段的说明，然后有一个上下文变量的占位符。

In [21]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [22]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [30]:
question = " 2023 年大语言模型效果最好的是哪个模型"

In [32]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

LLM 对问题的回答：" 我无法回答这个问题，因为我的训练数据只到 2023，之后的信息和事件我无法了解。感谢您的提问！"


这里因为没有对应的信息，所以大语言模型只能回答不知道。您可以将知识库的内容调整为大语言模型综述的内容重新进行尝试。

In [34]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='前言\n李宏毅老师是台湾大学的教授，其《机器学习》（2021 年春）是深度学习领域经典的中文视\n频之一。李老师幽默风趣的授课风格深受大家喜爱，让晦涩难懂的深度学习理论变得轻松易懂，\n他会通过很多动漫相关的有趣例子来讲解深度学习理论。李老师的课程内容很全面，覆盖了到深\n度学习必须掌握的常见理论，能让学生对于深度学习的绝大多数领域都有一定了解，从而可以进\n一步选择想要深入的方向进行学习，对于想入门深度学习又想看中文讲解的同学是非常推荐的。\n本教程主要内容源于《机器学习》（2021 年春），并在其基础上进行了一定的原创。比如，为\n了尽可能地降低阅读门槛，笔者对这门公开课的精华内容进行选取并优化，对所涉及的公式都给\n出详细的推导过程，对较难理解的知识点进行了重点讲解和强化，以方便读者较为轻松地入门。\n此外，为了丰富内容，笔者在教程中选取了《机器学习》（2017 年春） 的部分内容，并补充了不\n少除这门公开课之外的深度学习相关知识。\n致谢\n特别感谢 Sm1les、LSGOMYP 对本项目的帮助与支持。' metadata={'author': '', 'creationDate': "D:20230831225119+08'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../../docs/knowledge_base/LeeDL_Tutorial.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../../docs/knowledge_base/LeeDL_Tutorial.pdf', 'subject': '', 'title': '', 'total_pages': 330, 'trapped': ''}


这种方法非常好，因为它只涉及对语言模型的一次调用。然而，它也有局限性，即如果文档太多，可能无法将它们全部适配到上下文窗口中。

langchain 提供了几种不同的处理文档的方法：

|     类型      |                                定义/区别                                |                              优点                              |                              缺点                              |
|-------------|---------------------------------------------------------------------|----------------------------------------------------------------|----------------------------------------------------------------|
|   Stuff     | 将整个文本内容一次性输入给大模型进行处理。                               | - 只调用大模型一次，节省计算资源和时间。<br>- 上下文信息完整，有助于理解整体语义。<br>- 适用于处理较短的文本内容。 | - 不适用于处理较长的文本内容，可能导致模型过载。                |
|   Refine    | 通过多次调用大模型逐步改进文本质量，进行多次迭代优化。                          | - 可以在多次迭代中逐步改进文本质量。<br>- 适用于需要进行多次迭代优化的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的文本质量。<br>- 不适用于实时性要求较高的场景。 |
| Map reduce  | 将大模型应用于每个文档，并将输出作为新文档传递给另一个模型，最终得到单个输出。               | - 可以对多个文档进行并行处理，提高处理效率。<br>- 可以通过多次迭代处理实现优化。<br>- 适用于需要对多个文档进行处理和合并的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的结果。<br>- 不适用于处理单个文档的场景。 |
| Map re-rank | 在每个文档上运行初始提示，为答案给出一个分数，返回得分最高的响应。                        | - 可以根据置信度对文档进行排序和选择，提高结果的准确性。<br>- 可以提供更可靠的答案。<br>- 适用于需要根据置信度对文档进行排序和选择的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要对多个文档进行评分和排序。<br>- 不适用于不需要对文档进行排序和选择的场景。 |

我们可以根据需要配置 chain_type 的参数，选择对应的处理方式。如：
```
RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
```